<a href="https://colab.research.google.com/github/SoltanReza/classification/blob/dev/ProjetML2_2024_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prérequis

### Installation

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
import sys
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/ML_FDS'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd


!pip install umap-learn[plot]
!pip install holoviews
!pip install  ipykernel
!pip install joblib
# eventuellement ne pas oublier de relancer le kernel du notebook
# !pip install tensorflow==2.17.0

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/ML_FDS


In [ ]:
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.metrics import classification_report

# librairies générales
import joblib
import pickle
import pandas as pd
from scipy.stats import randint
import numpy as np
import string
import time
import base64
import re
import sys
import copy
import random
from numpy import mean
from numpy import std
import numpy as np

from google.colab import auth
import google.auth

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import plotly.graph_objs as go
import plotly.offline as py
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# TensorFlow et keras
import tensorflow as tf
from keras import layers
from keras import models
from keras import optimizers
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from tqdm import tqdm
from keras.models import load_model
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import os
from os import listdir
from os.path import isfile, join
import cv2
import glob
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io
import tempfile
from skimage.feature import hog
from sklearn.svm import SVC
import numpy as np



### Utils

In [ ]:
# afficher les res de train avec des plots ( version améliorée)
def plot_curves_confusion (history,confusion_matrix,class_names):
  plt.figure(1,figsize=(16,6))
  plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, right = 1,
                          top = 0.9, wspace = 0.25, hspace = 0)

  # division de la fenêtre graphique en 1 ligne, 3 colonnes,
  # graphique en position 1 - loss fonction

  plt.subplot(1,3,1)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['Training loss', 'Validation loss'], loc='upper left')
  # graphique en position 2 - accuracy
  plt.subplot(1,3,2)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['Training accuracy', 'Validation accuracy'], loc='upper left')

  # matrice de correlation
  plt.subplot(1,3,3)
  sns.heatmap(confusion_matrix,annot=True,fmt="d",cmap='Blues',xticklabels=class_names, yticklabels=class_names)# label=class_names)
  # labels, title and ticks
  plt.xlabel('Predicted', fontsize=12)
  #plt.set_label_position('top')
  #plt.set_ticklabels(class_names, fontsize = 8)
  #plt.tick_top()
  plt.title("Correlation matrix")
  plt.ylabel('True', fontsize=12)
  #plt.set_ticklabels(class_names, fontsize = 8)
  plt.show()



def plot_curves(histories):
    plt.figure(1,figsize=(16,6))
    plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, right = 1,
                          top = 0.9, wspace = 0.25, hspace = 0)
    for i in range(len(histories)):
    	# plot loss
    	plt.subplot(121)
    	plt.title('Cross Entropy Loss')
    	plt.plot(histories[i].history['loss'], color='blue', label='train')
    	plt.plot(histories[i].history['val_loss'], color='red', label='test')
    	plt.ylabel('loss')
    	plt.xlabel('epoch')
    	plt.legend(['Training loss', 'Validation loss'], loc='upper left')
    	# plot accuracy
    	plt.subplot(122)
    	plt.title('Classification Accuracy')
    	plt.ylabel('accuracy')
    	plt.xlabel('epoch')
    	plt.plot(histories[i].history['accuracy'], color='blue', label='train')
    	plt.plot(histories[i].history['val_accuracy'], color='red',
                 label='test')
    	plt.legend(['Training accuracy', 'Validation accuracy'], loc='upper left')
    plt.show()

# les fonctionnes d'authentification et enregistrer les résultat dans fichier csv
def authenticate_google_drive():
    """Authenticate and return the Google Drive API service."""
    auth.authenticate_user()  # This handles the OAuth2 authentication for Google Colab

    # Build the Google Drive API client
    service = build('drive', 'v3')
    return service

def download_file_from_drive(file_id):
    """Download a file from Google Drive."""
    service = authenticate_google_drive()

    # Request the file's content
    request = service.files().export_media(fileId=file_id, mimeType='text/csv')
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while done is False:
        status, done = downloader.next_chunk()

    fh.seek(0)
    return pd.read_csv(fh)

def upload_file_to_drive(file_id, file_path):
    """Upload the file to Google Drive after modification."""
    service = authenticate_google_drive()

    # Upload the modified file to Google Drive
    media = MediaFileUpload(file_path, mimetype='text/csv')
    updated_file = service.files().update(fileId=file_id, media_body=media).execute()

    print(f"File updated: {updated_file['name']}")

def append_train_results_to_csv(file_id, train_results):
    """Append new train results to an existing CSV file on Google Drive."""
    # Download the existing CSV file

    try:
        df = download_file_from_drive(file_id)
    except pd.errors.EmptyDataError:
        # If the file is empty, create an empty DataFrame with the correct columns
        column_names = list(train_results.keys())
        df = pd.DataFrame(columns=column_names)

    # Append the new results (train_results should be a dict)
    new_row = pd.DataFrame([train_results])
    df = pd.concat([df, new_row], ignore_index=True)

    # Save the modified DataFrame to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, mode='w', newline='') as tmpfile:
        temp_path = tmpfile.name
        df.to_csv(temp_path, index=False)

    # Upload the modified CSV back to Google Drive
    upload_file_to_drive(file_id, temp_path)

    # Bloc universel pour sauvegarder modèle et résultats
def save_model_and_results(model, model_name, x_train=None, y_train=None, x_val=None, y_val=None,
                           x_test=None, y_test=None, history=None, training_time=None,
                           batch_size=None, epochs=None, optimizer=None, file_id=None):

    # Identifiant unique basé sur la date/heure
    training_id = time.strftime("%Y%m%d%H%M%S")
    saved_model_filename = f"{model_name}_{training_id}.joblib"

    # Sauvegarder le modèle avec joblib
    model_path = f'/content/drive/MyDrive/models/{saved_model_filename}'
    joblib.dump(model, model_path)
    print(f'Modèle sauvegardé dans {model_path}')

    # Initialiser les métriques
    final_loss = None
    final_accuracy = None
    final_val_loss = None
    final_val_accuracy = None
    test_accuracy = None

    # Cas 1 : Modèles Keras avec history (CNN, ANN, etc.)
    if history is not None:
        final_loss = history.history.get('loss', [None])[-1]
        final_accuracy = history.history.get('accuracy', [None])[-1]
        final_val_loss = history.history.get('val_loss', [None])[-1]
        final_val_accuracy = history.history.get('val_accuracy', [None])[-1]

    # Cas 2 : Modèles scikit-learn (KNN, SVC, etc.) sans history
    else:
        if x_train is not None and y_train is not None:
            final_accuracy = model.score(x_train, y_train) if hasattr(model, 'score') else None
        if x_val is not None and y_val is not None:
            y_pred_val = model.predict(x_val)
            final_val_accuracy = accuracy_score(y_val, y_pred_val)
        if x_test is not None and y_test is not None:
            y_pred_test = model.predict(x_test)
            test_accuracy = accuracy_score(y_test, y_pred_test)

    # Remplir le dictionnaire des résultats
    train_results = {
        'Training ID': training_id,
        'Model Name': model_name,
        'Coach': 'Reza',
        'Epoch': epochs if epochs else None,
        'Loss': final_loss,
        'Accuracy': final_accuracy,
        'Validation Loss': final_val_loss,
        'Validation Accuracy': final_val_accuracy,
        'Test Accuracy': test_accuracy,
        'Training Time (s)': training_time if training_time else None,
        'Batch Size': batch_size if batch_size else None,
        'Learning Rate': None,  # À ajouter manuellement si pertinent
        'Optimizer': optimizer if optimizer else None,
        'Model Type': 'Sequential' if history else 'Scikit-learn',
        'Dataset Type': 'Custom',
        'Saved Model Filename': saved_model_filename
    }

    # Sauvegarder dans Google Drive
    if file_id:
        append_train_results_to_csv(file_id, train_results)
    else:
        print("Aucun file_id fourni, résultats non sauvegardés dans Google Drive.")


# Fonction générique pour visualiser les résultats
def plot_model_results(model, model_name, x_test, y_test, history=None, class_names=None):
    """
        Args:
        model: Modèle entraîné (Keras ou scikit-learn)
        model_name: Nom du modèle (str)
        x_test: Données de test
        y_test: Étiquettes réelles de test
        history: Objet history de Keras (optionnel)
        class_names: Liste des noms de classes (ex. ['Non-tigre', 'Tigre'])
    """
    # Prédictions
    if 'tensorflow' in str(type(model)):  # Modèles Keras
        y_pred = model.predict(x_test)
        # Si y_pred est un tableau 2D (comme pour CNN avec sigmoid), le convertir en classes
        if len(y_pred.shape) > 1 and y_pred.shape[1] == 1:
            y_pred_classes = (y_pred > 0.5).astype(int).flatten()
        else:
            y_pred_classes = y_pred.argmax(axis=1)  # Pour multiclasse
    else:  # Modèles scikit-learn
        y_pred_classes = model.predict(x_test)

    # Calcul de la matrice de confusion
    cm = confusion_matrix(y_test, y_pred_classes)

    # Configuration de la figure
    if history:  # Si history existe, 3 sous-graphiques (loss, accuracy, confusion)
        plt.figure(figsize=(16, 6))
        plt.gcf().subplots_adjust(left=0.125, bottom=0.2, right=1, top=0.9, wspace=0.25, hspace=0)
        num_plots = 3
    else:  # Sinon, seulement la matrice de confusion
        plt.figure(figsize=(6, 6))
        num_plots = 1

    # Courbes d’apprentissage (si history existe)
    if history:
        # Loss
        plt.subplot(1, 3, 1)
        plt.plot(history.history['loss'], label='Training loss')
        plt.plot(history.history['val_loss'], label='Validation loss')
        plt.title(f'{model_name} - Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(loc='upper left')

        # Accuracy
        plt.subplot(1, 3, 2)
        plt.plot(history.history['accuracy'], label='Training accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation accuracy')
        plt.title(f'{model_name} - Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(loc='upper left')

        # Matrice de confusion
        plt.subplot(1, 3, 3)
    else:
        # Seulement la matrice de confusion
        plt.subplot(1, 1, 1)

    # Affichage de la matrice de confusion
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues',
                xticklabels=class_names if class_names else ['Class 0', 'Class 1'],
                yticklabels=class_names if class_names else ['Class 0', 'Class 1'])
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    plt.show()


### Les jeux de données


In [ ]:
!wget https://www.lirmm.fr/~poncelet/Ressources/Tiger-Fox-Elephant.zip

--2025-03-06 14:11:30--  https://www.lirmm.fr/~poncelet/Ressources/Tiger-Fox-Elephant.zip
Resolving www.lirmm.fr (www.lirmm.fr)... 193.49.104.251
Connecting to www.lirmm.fr (www.lirmm.fr)|193.49.104.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7605545 (7.3M) [application/zip]
Saving to: ‘Tiger-Fox-Elephant.zip.18’

Tiger-Fox-Elephant. 100%[===================>]   7.25M  4.75MB/s    in 1.5s    

2025-03-06 14:11:33 (4.75 MB/s) - ‘Tiger-Fox-Elephant.zip.18’ saved [7605545/7605545]



In [ ]:

import zipfile
#with zipfile.ZipFile("Tiger-Fox-Elephant.zip","r") as zip_ref:
    #zip_ref.extractall("Data_Project")

**Créer le jeu de données**

Actuellement pour chaque animal nous avons un répertoire qui contient des images positives et un répertoire qui contient des images négatives. Pour pouvoir créer un jeu de données nous devons obtenir X et y. Les fonctions ci-dessous permettent de générer, à partir des répertoires, un jeu de données aléatoire pour X et y.

In [ ]:
def create_training_data(path_data, list_classes):
  training_data=[]
  for classes in list_classes:
      path=os.path.join(path_data, classes)
      class_num=list_classes.index(classes)
      for img in os.listdir(path):
        try:
          img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_UNCHANGED)
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
          training_data.append([new_array, class_num])
        except Exception as e:
          pass
  return training_data

def create_X_y (path_data, list_classes):
      # récupération des données
      training_data=create_training_data(path_data, list_classes)
      # tri des données
      random.shuffle(training_data)
      # création de X et y
      X=[]
      y=[]
      for features, label in training_data:
        X.append(features)
        y.append(label)
      X=np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
      y=np.array(y)
      return X,y

def plot_examples(X,y):
  plt.figure(figsize=(15,15))
  for i in range(COLUMNS):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # cv2 lit met les images en BGR et matplotlib lit du RGB
    X[i] = cv2.cvtColor(X[i], cv2.COLOR_BGR2RGB)
    plt.imshow(X[i]/255.,cmap=plt.cm.binary)
    plt.xlabel('classe ' + str(y[i]))

# Train


## Preparations

In [ ]:
# constantes globales

IMG_SIZE=64
COLUMNS = 25 # Nombre d'images à afficher
my_path="Data_Project/Tiger-Fox-Elephant/"
my_classes=['tiger','Tiger_negative_class']

#Pré-requis CNN et ANN
optimizer = 'adam'
activation='relu'
batch_size=32
epochs=15

# Charger les données
X, y =create_X_y(my_path,my_classes)
X=X.astype('float')
X=X/255.0
# Diviser en ensembles d'entraînement, de validation, et de test
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

## Modèles

In [ ]:

#Pré-requis
optimizer = 'adam'
activation='relu'
batch_size=32
epochs=15
model_name="CNN"
print(f"Train: {len(x_train)}, Validation: {len(x_val)}, Test: {len(x_test)}")

# === 2. Construire le modèle CNN ===
model = models.Sequential([
    # Première couche de convolution
    layers.Conv2D(32, (3, 3), activation=activation, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),

    # Deuxième couche de convolution
    layers.Conv2D(64, (3, 3), activation=activation),
    layers.MaxPooling2D((2, 2)),

    # Troisième couche de convolution
    layers.Conv2D(128, (3, 3), activation=activation),
    layers.MaxPooling2D((2, 2)),

    # Aplatir les données et ajouter les couches denses
    layers.Flatten(),
    layers.Dense(128, activation=activation),
    # layers.Dropout(0.5),  # Pour éviter le surapprentissage
    layers.Dense(1, activation='sigmoid')  # Sigmoïde pour classification binaire
])

# === 3. Compiler et entraîner le modèle ===
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',  # Perte pour la classification binaire
              metrics=['accuracy'])

print(model.summary())  # Résumé du modèle
start_time = time.time()

# Entraîner le modèle
history = model.fit(
    x_train, y_train,
    epochs=epochs,
    validation_data=(x_val, y_val),
    batch_size=batch_size
)
training_time = time.time() - start_time

#évaluer performances
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


##Modèle + KFold

In [ ]:

model_name="CNN-KFOLD"



# Paramètres pour k-fold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Liste pour stocker les accuracies de chaque fold
accuracies_cnn = []

# Fonction pour créer le modèle CNN
def create_cnn():
    model = models.Sequential([
        # Première couche de convolution
        layers.Conv2D(32, (3, 3), activation=activation, input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        layers.MaxPooling2D((2, 2)),

        # Deuxième couche de convolution
        layers.Conv2D(64, (3, 3), activation=activation),
        layers.MaxPooling2D((2, 2)),

        # Troisième couche de convolution
        layers.Conv2D(128, (3, 3), activation=activation),
        layers.MaxPooling2D((2, 2)),

        # Aplatir les données et ajouter les couches denses
        layers.Flatten(),
        layers.Dense(128, activation=activation),
        # layers.Dropout(0.5),  # Décommenter si tu veux éviter le surapprentissage
        layers.Dense(1, activation='sigmoid')  # Sigmoïde pour classification binaire
    ])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Validation croisée
for fold, (train_index, val_index) in enumerate(kf.split(x_train)):
    x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    print(f"\nFold {fold+1}/{k}")
    print(f"Train: {len(x_train_fold)}, Validation: {len(x_val_fold)}")

    # Créer et entraîner le modèle
    model = create_cnn()
    start_time = time.time()
    history = model.fit(
        x_train_fold, y_train_fold,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_val_fold, y_val_fold),
        verbose=0  # Silence pour éviter trop d’affichage
    )
    training_time = time.time() - start_time

    # Évaluation sur le fold de validation (dernière epoch)
    val_loss, val_acc = model.evaluate(x_val_fold, y_val_fold, verbose=0)
    accuracies_cnn.append(val_acc)
    print(f"Fold {fold+1} - Validation Accuracy: {val_acc * 100:.2f}%, Training Time: {training_time:.2f}s")

# Calcul de la moyenne et de l’écart type
mean_acc = np.mean(accuracies_cnn)
std_acc = np.std(accuracies_cnn)
print(f"\nCNN - Moyenne Accuracy (k-fold): {mean_acc * 100:.2f}%")
print(f"CNN - Écart type Accuracy (k-fold): {std_acc * 100:.2f}%")

# # Évaluation finale sur le jeu de test (optionnel)
# model_final = create_cnn()
# model_final.fit(x_kfold, y_kfold, epochs=epochs, batch_size=batch_size, verbose=0)
# test_loss, test_accuracy = model_final.evaluate(x_test, y_test, verbose=0)
# print(f"CNN - Test Accuracy (sur jeu de test séparé): {test_accuracy * 100:.2f}%")

NameError: name 'KFold' is not defined

## Visualiser les résultat

In [ ]:


y_pred = model.predict(x_test)  # Sortie brute du modèle
y_pred_classes = []
for i in y_pred:
    for j in i:
      y_pred_classes.append ((j > 0.5).astype(int)) # Convert to class 0 or 1


cm = confusion_matrix(y_test , y_pred_classes)
plot_model_results(model=model,
    model_name=model_name,
    x_test=x_test,
    y_test=y_test,
    history=history ? history : None,
    class_names=my_classes)

NameError: name 'model' is not defined

## Sauvegarder les modèles et Enregistrer les résultats

In [ ]:

# Récupérer les résultats de l'entraînement
final_loss = history.history['loss'][-1] or None
final_accuracy = history.history['accuracy'][-1] or None
final_val_loss = history.history['val_loss'][-1] or None
final_val_accuracy = history.history['val_accuracy'][-1] or None
training_id = time.strftime("%Y%m%d%H%M%S")
# Scopes required to access and modify Google Drive files
SCOPES = ['https://www.googleapis.com/auth/drive.file']




# La bibliothèque joblib et donc dump a déjà été importée
saved_model_filename = training_id
# Sauvegarder le modèle (clf) en utilisant joblib

joblib.dump(model,'models/' + saved_model_filename ) # Pas besoin d'ouvrir le fichier en mode binaire
# Confirmation que le modèle a été sauvegardé
print(f'Modèle sauvegardé dans models/{saved_model_filename}')


# Example usage
file_id = '1sEaqLjJN7lZCKJ2fYbOmiqJIytB8wIpKChZkK32kzCI'  # Replace with your actual Google Drive file ID
# Remplir le template avec ces résultats
train_results = {
    'Training ID': training_id,  # Timestamp comme ID d'entraînement
    'Model Name': model_name,  # Nom du modèle
    'Coach' : 'Reza',
    'Epoch': epochs,  # Nombre d'époques
    'Loss': final_loss,
    'Accuracy': final_accuracy,
    'Validation Loss': final_val_loss,
    'Validation Accuracy': final_val_accuracy,
    'Training Time (s)': training_time,
    'Batch size': batch_size,
    'Learning Rate': None,  # Exemples de valeurs pour le learning rate
    'Optimizer': optimizer,  # Optimiseur utilisé
    'Model Type': 'Sequential',  # Type de modèle
    'Dataset Type': 'Custom' , # Type de dataset
    'Saved model filename': saved_model_filename
}

append_train_results_to_csv(file_id, train_results)


In [ ]:
# Bloc universel pour sauvegarder modèle et résultats
def save_model_and_results(model, model_name, x_train=None, y_train=None, x_val=None, y_val=None,
                           x_test=None, y_test=None, history=None, training_time=None,
                           batch_size=None, epochs=None, optimizer=None, file_id=None):

    # Identifiant unique basé sur la date/heure
    training_id = time.strftime("%Y%m%d%H%M%S")
    saved_model_filename = f"{model_name}_{training_id}.joblib"

    # Sauvegarder le modèle avec joblib
    model_path = f'/content/drive/MyDrive/models/{saved_model_filename}'
    joblib.dump(model, model_path)
    print(f'Modèle sauvegardé dans {model_path}')

    # Initialiser les métriques
    final_loss = None
    final_accuracy = None
    final_val_loss = None
    final_val_accuracy = None
    test_accuracy = None

    # Cas 1 : Modèles Keras avec history (CNN, ANN, etc.)
    if history is not None:
        final_loss = history.history.get('loss', [None])[-1]
        final_accuracy = history.history.get('accuracy', [None])[-1]
        final_val_loss = history.history.get('val_loss', [None])[-1]
        final_val_accuracy = history.history.get('val_accuracy', [None])[-1]

    # Cas 2 : Modèles scikit-learn (KNN, SVC, etc.) sans history
    else:
        if x_train is not None and y_train is not None:
            final_accuracy = model.score(x_train, y_train) if hasattr(model, 'score') else None
        if x_val is not None and y_val is not None:
            y_pred_val = model.predict(x_val)
            final_val_accuracy = accuracy_score(y_val, y_pred_val)
        if x_test is not None and y_test is not None:
            y_pred_test = model.predict(x_test)
            test_accuracy = accuracy_score(y_test, y_pred_test)

    # Remplir le dictionnaire des résultats
    train_results = {
        'Training ID': training_id,
        'Model Name': model_name,
        'Coach': 'Reza',
        'Epoch': epochs if epochs else None,
        'Loss': final_loss,
        'Accuracy': final_accuracy,
        'Validation Loss': final_val_loss,
        'Validation Accuracy': final_val_accuracy,
        'Test Accuracy': test_accuracy,
        'Training Time (s)': training_time if training_time else None,
        'Batch Size': batch_size if batch_size else None,
        'Learning Rate': None,  # À ajouter manuellement si pertinent
        'Optimizer': optimizer if optimizer else None,
        'Model Type': 'Sequential' if history else 'Scikit-learn',
        'Dataset Type': 'Custom',
        'Saved Model Filename': saved_model_filename
    }

    # Sauvegarder dans Google Drive
    if file_id:
        append_train_results_to_csv(file_id, train_results)
    else:
        print("Aucun file_id fourni, résultats non sauvegardés dans Google Drive.")